# Plotting the DC Values vs. Time

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/combined_LLPSA2.csv')
all_experiments = data['experiment_id'].unique()
all_experiments.sort()
dc_columns = [col for col in data.columns if col.startswith('DC_')]
ac_columns = [col for col in data.columns if col.startswith('Amp_')]

# Compute AC by DC
for idx in range(len(ac_columns)):
    ac_col = ac_columns[idx]
    dc_col = dc_columns[idx]
    data[f"EPR {idx}"] = data[ac_col].to_numpy() / data[dc_col].to_numpy()
    
epr_columns = [f"EPR {idx}" for idx in range(len(ac_columns))]

In [ ]:
from ipywidgets import interact, Dropdown
import matplotlib.pyplot as plt

col_to_plot_options = ["DC", "AC", "EPR", "Log EPR"]
color_mapping = {
    "DC": "blue",
    "AC": "orange",
    "EPR": "red",
    "Log EPR": "purple",
    "Saturation": "green"
}


def plot_dc_values(experiment_id, col_to_plot="DC"):
    assert col_to_plot in col_to_plot_options, "Unknown col_to_plot"
    # Filter data for selected experiment
    exp_data = data[data['experiment_id'] == experiment_id]
    saturation = exp_data['fSaO2'].to_numpy()
    
    # Create 6x2 subplot grid
    fig, axes = plt.subplots(6, 2, figsize=(7, 8))
    
    # Plot each DC column
    for i in range(len(dc_columns)):
        dc_col = dc_columns[i]
        ac_col = ac_columns[i]
        epr_col = epr_columns[i]
        if col_to_plot == "DC":
            axes[i % 5, i // 5].plot(exp_data.index, exp_data[dc_col], color=color_mapping["DC"])
        elif col_to_plot == "AC":
            axes[i % 5, i // 5].plot(exp_data.index, exp_data[ac_col], color=color_mapping["AC"])
        elif col_to_plot == "EPR":
            axes[i % 5, i // 5].plot(exp_data.index, exp_data[epr_col], color=color_mapping["EPR"])
        else:
            axes[i % 5, i // 5].plot(exp_data.index, np.log(exp_data[epr_col]), color=color_mapping["Log EPR"])
        axes[i % 5, i // 5].set_title(f'{col_to_plot} - Column {i}')
        axes[i % 5, i // 5].set_xlabel('Index')
        axes[i % 5, i // 5].set_ylabel(f'{col_to_plot} Value')
        axes[i % 5, i // 5].grid(True, alpha=0.3)
    
    # Plot saturation in the second last subplot
    axes[-1, 0].plot(exp_data.index, saturation, color=color_mapping["Saturation"])
    axes[-1, 0].set_title('Saturation (fSaO2)')
    axes[-1, 0].set_xlabel('Index')
    axes[-1, 0].set_ylabel('Saturation')
    axes[-1, 0].grid(True, alpha=0.3)
    axes[-1, 1].axis('off')  # Hide the last subplot if unused
    
    plt.tight_layout()
    plt.show()

# Create dropdown widget with all experiments
interact(plot_dc_values, experiment_id=Dropdown(options=all_experiments, description='Round:'), col_to_plot=Dropdown(options=["DC", "AC", "EPR", "Log EPR"], description='Type:'))

interactive(children=(Dropdown(description='Round:', options=('sp2021_S1R2', 'sp2021_S1R3', 'sp2021_S2R1', 'sp…

<function __main__.plot_dc_values(experiment_id, col_to_plot='DC')>